# Sentiment analysis model

In [19]:
import pandas , numpy , sklearn
import tensorflow as tf
import matplotlib.pyplot as plt
import torch


tf.Tensor(
[[102 126 150]
 [138 171 204]], shape=(2, 3), dtype=int32)


3 